# last.fm Data Exploraton - Top Albums

In [1]:
# Defining API KEY as variable
API_KEY = '2357e38959224b0d01d286b5993f972c'

# Defining USER_AGENT as a variable
USER_AGENT = 'recatoma'

# Defining Shared Secred as a variable
SHARED_SECRET = '52a457c8fd0c979cd8d267188d7e0523'


In [2]:
# Importing requests package

import requests

In [3]:
#  Specifying user-agent header to idenfity user
headers = {
    'user-agent': USER_AGENT
}

# Specifying the API key and table we want to get from the API
payload = {
    'api_key': API_KEY,
    'method': 'user.getTopAlbums',
    'user' : 'recatoma',
    'format': 'json'
}

In [4]:
# Defining function to get the API data 
def lastfm_get(payload):
    # Define headers and URL
    headers = {'user-agent': USER_AGENT}
    url = 'http://ws.audioscrobbler.com/2.0/'

    # Add API key and format to the payload
    payload['api_key'] = API_KEY
    payload['format'] = 'json'
    payload['user'] = 'recatoma'
    payload['limit'] = 50
    response = requests.get(url, headers=headers, params=payload)
    return response

In [5]:
# Running the function to get the Top Artists table
r = lastfm_get({
    'method': 'user.getTopAlbums'
})

# Return status of the request
r.status_code

200

In [6]:
# Importing json module
import json

# Creating function to print response from the API
def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

jprint(r.json())

{
    "topalbums": {
        "@attr": {
            "page": "1",
            "perPage": "50",
            "total": "1242",
            "totalPages": "25",
            "user": "recatoma"
        },
        "album": [
            {
                "@attr": {
                    "rank": "1"
                },
                "artist": {
                    "mbid": "9e53f84d-ef44-4c16-9677-5fd4d78cbd7d",
                    "name": "Wilco",
                    "url": "https://www.last.fm/music/Wilco"
                },
                "image": [
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/34s/3fbe4bba18f1482db78b770f7e8fa593.jpg",
                        "size": "small"
                    },
                    {
                        "#text": "https://lastfm.freetls.fastly.net/i/u/64s/3fbe4bba18f1482db78b770f7e8fa593.jpg",
                        "size": "medium"
                    },
                    {
                        "#tex

In [7]:
# Checking the first dictionary of the table

jprint(r.json()['topalbums']['@attr'])

{
    "page": "1",
    "perPage": "50",
    "total": "1242",
    "totalPages": "25",
    "user": "recatoma"
}


In [8]:
# Importing requests-cache library
import requests_cache

# Function to cache the API requests
requests_cache.install_cache()

In [9]:
# Redefining function to get the API data 
def lastfm_get(payload):
    # Define headers and URL
    headers = {'user-agent': USER_AGENT}
    url = 'http://ws.audioscrobbler.com/2.0/'

    # Add API key and format to the payload
    payload['api_key'] = API_KEY
    payload['format'] = 'json'

    response = requests.get(url, headers=headers, params=payload)
    return response

In [10]:
import time
from IPython.core.display import clear_output

# Empty list to store the API content
responses = []

page = 1
total_pages = 99999 # This is just a dummy number so the loop starts

while page <= total_pages:
    payload = {
        'method': 'user.getTopAlbums',
        'user' : 'recatoma',
        'limit': 50,
        'page': page
    }
    # Print some output so we can see the status
    print("Requesting page {}/{}".format(page, total_pages))
    # Clear the output to make things neater
    clear_output(wait = True)

    # Make the API call
    response = lastfm_get(payload)

    # If we get an error, print the response and halt the loop
    if response.status_code != 200:
        print(response.text)
        break

    # Extract pagination info
    page = int(response.json()['topalbums']['@attr']['page'])
    total_pages = int(response.json()['topalbums']['@attr']['totalPages'])

    # Append response
    responses.append(response)

    # If it's not a cached result, sleep
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)

    # Increment the page number
    page += 1

Requesting page 25/25


In [11]:
import pandas as pd

# List comprehension
frames = [pd.DataFrame(r.json()['topalbums']['album']) for r in responses]

# Creating the DataFrame by using the concatenate method
recatoma_albums = pd.concat(frames)

# Visualizing the DataFrame
recatoma_albums.head()

,artist,@attr,image,playcount,url,name,mbid
0,"{'url': 'https://www.last.fm/music/Wilco', 'na...",{'rank': '1'},"[{'size': 'small', '#text': 'https://lastfm.fr...",72,https://www.last.fm/music/Wilco/Sky+Blue+Sky,Sky Blue Sky,17714744-82eb-424f-b993-21d34efb6c3c
1,{'url': 'https://www.last.fm/music/The+United+...,{'rank': '2'},"[{'size': 'small', '#text': 'https://lastfm.fr...",60,https://www.last.fm/music/The+United+States+of...,The United States of America,48134939-3bfb-4148-9916-2ec5f1ff136a
2,{'url': 'https://www.last.fm/music/The+Fiery+F...,{'rank': '3'},"[{'size': 'small', '#text': 'https://lastfm.fr...",57,https://www.last.fm/music/The+Fiery+Furnaces/W...,Widow City,50401dd2-49a4-4be8-98b1-00a611e501de
3,"{'url': 'https://www.last.fm/music/The+The', '...",{'rank': '4'},"[{'size': 'small', '#text': 'https://lastfm.fr...",55,https://www.last.fm/music/The+The/Mind+Bomb,Mind Bomb,492ab79b-97cc-47f1-9a51-877059c128e4
4,"{'url': 'https://www.last.fm/music/Feist', 'na...",{'rank': '5'},"[{'size': 'small', '#text': 'https://lastfm.fr...",53,https://www.last.fm/music/Feist/Let+It+Die,Let It Die,279534e7-b1f4-4031-8a11-8838d4844564


In [12]:
recatoma_albums.iloc[40:70]

,artist,@attr,image,playcount,url,name,mbid
40,{'url': 'https://www.last.fm/music/My+Bloody+V...,{'rank': '41'},"[{'size': 'small', '#text': 'https://lastfm.fr...",26,https://www.last.fm/music/My+Bloody+Valentine/...,Loveless,09a09037-1ebd-3d38-a128-38ab11e6b0fe
41,"{'url': 'https://www.last.fm/music/Pinback', '...",{'rank': '42'},"[{'size': 'small', '#text': 'https://lastfm.fr...",26,https://www.last.fm/music/Pinback/Blue+Screen+...,Blue Screen Life,4c6c7162-7b1b-48b2-b0c2-a31a15d0c889
42,{'url': 'https://www.last.fm/music/Super+Furry...,{'rank': '43'},"[{'size': 'small', '#text': 'https://lastfm.fr...",26,https://www.last.fm/music/Super+Furry+Animals/...,Radiator,0e6dcdec-1f86-3b8d-bd94-dc4a649ccc1e
43,{'url': 'https://www.last.fm/music/Talking+Hea...,{'rank': '44'},"[{'size': 'small', '#text': 'https://lastfm.fr...",26,https://www.last.fm/music/Talking+Heads/Speaki...,Speaking in Tongues,1932169a-607a-46c4-a611-3a8479c1b885
44,{'url': 'https://www.last.fm/music/Digitalism'...,{'rank': '45'},"[{'size': 'small', '#text': 'https://lastfm.fr...",25,https://www.last.fm/music/Digitalism/Idealism,Idealism,216ff204-4b4b-437a-a039-8b603a005cbe
45,{'url': 'https://www.last.fm/music/Marisa+Mont...,{'rank': '46'},"[{'size': 'small', '#text': 'https://lastfm.fr...",25,https://www.last.fm/music/Marisa+Monte/Verde+a...,Verde anil amarelo cor de rosa e carvão,5d18e03e-e9cc-426f-b1aa-7d74028317db
46,{'url': 'https://www.last.fm/music/+noredirect...,{'rank': '47'},"[{'size': 'small', '#text': 'https://lastfm.fr...",25,https://www.last.fm/music/+noredirect/Nick+Cav...,Abattoir Blues,
47,{'url': 'https://www.last.fm/music/Talking+Hea...,{'rank': '48'},"[{'size': 'small', '#text': 'https://lastfm.fr...",25,https://www.last.fm/music/Talking+Heads/Fear+o...,Fear of Music,00f09140-c4d5-40fe-9256-cee44fc1a6d6
48,{'url': 'https://www.last.fm/music/The+Free+De...,{'rank': '49'},"[{'size': 'small', '#text': 'https://lastfm.fr...",25,https://www.last.fm/music/The+Free+Design/You+...,You Could Be Born Again,33ccee1c-9187-4e7f-b96e-04bec0549fb6
49,{'url': 'https://www.last.fm/music/The+Ladybug...,{'rank': '50'},"[{'size': 'small', '#text': 'https://lastfm.fr...",25,https://www.last.fm/music/The+Ladybug+Transist...,The Albemarle Sound,55a67309-e1b6-35b0-9b69-ffb4dbcfa86b


In [13]:
recatoma_albums.reset_index(inplace=True)

In [14]:
# Transforming the dictionarys contained in each row of the 'artist' column into a list
artist_df = pd.DataFrame(recatoma_albums['artist'].values.tolist())

In [15]:
artist_df.iloc[45:70]

,url,name,mbid
45,https://www.last.fm/music/Marisa+Monte,Marisa Monte,f81f19b9-c76e-43ac-8656-bb56071785fb
46,https://www.last.fm/music/+noredirect/Nick+Cav...,Nick Cave and the Bad Seeds,
47,https://www.last.fm/music/Talking+Heads,Talking Heads,a94a7155-c79d-4409-9fcf-220cb0e4dc3a
48,https://www.last.fm/music/The+Free+Design,The Free Design,8e2188f5-c28b-4cf5-9f01-b3ed0ed2b00b
49,https://www.last.fm/music/The+Ladybug+Transistor,The Ladybug Transistor,034147e1-0acd-4a3a-9fcf-aa310cb41707
50,https://www.last.fm/music/Girl+Talk,Girl Talk,
51,https://www.last.fm/music/Stars,Stars,5aeb21a3-4606-4a0d-8369-50c768d1e99a
52,https://www.last.fm/music/Joan+as+Police+Woman,Joan as Police Woman,62773859-b9b9-4108-b814-cfd91459d716
53,https://www.last.fm/music/Joshua+Redman,Joshua Redman,
54,https://www.last.fm/music/Kinky,Kinky,21042d7e-4622-45f0-b1a0-a6dfa458a2e3


In [16]:
# Assigning the name of the artists in the 'artist'column 
recatoma_albums['artists'] = artist_df['name']

In [17]:
artist_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1210 entries, 0 to 1209
Data columns (total 3 columns):
url     1210 non-null object
name    1210 non-null object
mbid    1210 non-null object
dtypes: object(3)
memory usage: 28.5+ KB


In [18]:
# Visualizing changes
recatoma_albums.head()

,index,artist,@attr,image,playcount,url,name,mbid,artists
0,0,"{'url': 'https://www.last.fm/music/Wilco', 'na...",{'rank': '1'},"[{'size': 'small', '#text': 'https://lastfm.fr...",72,https://www.last.fm/music/Wilco/Sky+Blue+Sky,Sky Blue Sky,17714744-82eb-424f-b993-21d34efb6c3c,Wilco
1,1,{'url': 'https://www.last.fm/music/The+United+...,{'rank': '2'},"[{'size': 'small', '#text': 'https://lastfm.fr...",60,https://www.last.fm/music/The+United+States+of...,The United States of America,48134939-3bfb-4148-9916-2ec5f1ff136a,The United States of America
2,2,{'url': 'https://www.last.fm/music/The+Fiery+F...,{'rank': '3'},"[{'size': 'small', '#text': 'https://lastfm.fr...",57,https://www.last.fm/music/The+Fiery+Furnaces/W...,Widow City,50401dd2-49a4-4be8-98b1-00a611e501de,The Fiery Furnaces
3,3,"{'url': 'https://www.last.fm/music/The+The', '...",{'rank': '4'},"[{'size': 'small', '#text': 'https://lastfm.fr...",55,https://www.last.fm/music/The+The/Mind+Bomb,Mind Bomb,492ab79b-97cc-47f1-9a51-877059c128e4,The The
4,4,"{'url': 'https://www.last.fm/music/Feist', 'na...",{'rank': '5'},"[{'size': 'small', '#text': 'https://lastfm.fr...",53,https://www.last.fm/music/Feist/Let+It+Die,Let It Die,279534e7-b1f4-4031-8a11-8838d4844564,Feist


In [19]:
# Dropping columns that won't be used here

recatoma_albums.drop(['@attr', 'mbid', 'url', 'image', 'index', 'artist'], axis =1, inplace = True)

In [20]:
# Stats from the DataFrame

recatoma_albums.info()
recatoma_albums.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1210 entries, 0 to 1209
Data columns (total 3 columns):
playcount    1210 non-null object
name         1210 non-null object
artists      1210 non-null object
dtypes: object(3)
memory usage: 28.5+ KB


,playcount,name,artists
count,1210,1210,1210
unique,47,1154,775
top,1,Pitchfork Top 200 Songs of the 2000s,Caetano Veloso
freq,623,32,21


In [21]:
# Function to get all top 5 tags for each artist
def lookup_tags(artist,album):
    response = lastfm_get({
        'method': 'album.getTopTags',
        'artist': artist,
        'album': album,
        'autocorrect':1
    })
    
    # If there's an error, return nothing
    if response.status_code != 200:
        return None
    
    # Extract the top 5 tags and turn them into a string
    tags = [t['name'] for t in response.json()['toptags']['tag'][:5]]
    tags_str = ", ".join(tags)
    
    # Rate limiting
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)
    
    return tags_str

In [22]:
# Testing the function

lookup_tags("Black Sabbath","Paranoid")

'heavy metal, hard rock, albums I own, classic rock, 70s'

In [25]:
# Correcting name of an artist

recatoma_albums['artists'].iloc[186] = 'The Replacements' 

In [26]:
# Importing itertools package
import itertools

# List with the name of the artists
artists = recatoma_albums['artists'].to_list()

# List with the name of the albums
albums = recatoma_albums['name'].to_list()

In [38]:
# Empty list to store the tags
tags = []
counter = 1

for i, j in zip(artists, albums):
    try:
        tags.append(lookup_tags(i,j))
    except KeyError:
        tags.append(" ")
    print(counter, end=' ')
    counter += 1

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [39]:
# Visualizing the list
tags

['albums I own, best of 2007, indie, 2007, favorite albums',
 'psychedelic, 60s, experimental, 1968, Psychedelic Rock',
 'albums I own, indie, american, 00s, best of 2007',
 '1989, alternative, 80s, johnny marr, never stale',
 'albums I own, female vocalists, favorite albums, indie, 2004',
 'albums I own, best of 2007, indie rock, 2007, indie',
 'electronic, albums I own, electro, french, 2007',
 'indie, indie rock, favorite albums, albums I own, rock',
 '1999, experimental, psychedelic pop, psychedelic, Lo-Fi',
 'mpb, clube da esquina, brazilian, milton nascimento, brazil',
 'soul, motown, 70s, funk, albums I own',
 '2019',
 'me faz feliz, dormindo na praia',
 'alternative, 80s, 1988, icelandic, alternative rock',
 'electronic, House, albums I own, dance, electronica',
 '80s, 1983, albums I own, rock, pop',
 'shoegazegdchill, sexy, female vocalists, Erotic, redhot',
 'albums I own, los hermanos, rock, alternative, indie rock',
 'albums I own, alternative rock, rock, 90s, 1995',
 'albu

In [40]:
# Adding a columns for the tags into the DataFrame

recatoma_albums['tags'] = tags

In [43]:
# Visualizing the DataFrame

recatoma_albums.head(30)

,playcount,name,artists,tags
0,72,Sky Blue Sky,Wilco,"albums I own, best of 2007, indie, 2007, favor..."
1,60,The United States of America,The United States of America,"psychedelic, 60s, experimental, 1968, Psychede..."
2,57,Widow City,The Fiery Furnaces,"albums I own, indie, american, 00s, best of 2007"
3,55,Mind Bomb,The The,"1989, alternative, 80s, johnny marr, never stale"
4,53,Let It Die,Feist,"albums I own, female vocalists, favorite album..."
5,50,Ga Ga Ga Ga Ga,Spoon,"albums I own, best of 2007, indie rock, 2007, ..."
6,48,Cross,Justice,"electronic, albums I own, electro, french, 2007"
7,48,Summer in Abaddon,Pinback,"indie, indie rock, favorite albums, albums I o..."
8,48,Black Foliage: Animation Music,The Olivia Tremor Control,"1999, experimental, psychedelic pop, psychedel..."
9,47,Clube Da Esquina,Milton Nascimento,"mpb, clube da esquina, brazilian, milton nasci..."


In [42]:
# Info about the last version of the DataFrame

recatoma_albums.iloc[]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1210 entries, 0 to 1209
Data columns (total 4 columns):
playcount    1210 non-null object
name         1210 non-null object
artists      1210 non-null object
tags         1210 non-null object
dtypes: object(4)
memory usage: 37.9+ KB


In [44]:
# Exporting the DataFrame to csv

recatoma_albums.to_csv('recatoma_albums.csv', index = None, header=True)